### Lab 2: Hyponyms and Hypernyms

In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
%matplotlib inline

4


In [2]:
df = pd.read_csv('nlp2.csv', sep = '|', encoding='utf-8')
df.head()

,HYPONYM,HYPERNYM,TEXT,LINK
0,ПАПЕРТЬ,внешний притвор,"Па́перть, или внешний притвор (греч. ἄτριον, л...",https://ru.wikipedia.org/wiki/Паперть
1,ПАПЕРТЬ,площадка перед церковью,Паперть — площадка перед церковью и крыльцо.,http://window.edu.ru/catalog/pdf2txt/568/25568...
2,ПАПЕРТЬ,церковное крыльцо,"Церковное крыльцо, площадка перед входом в цер...",https://slovar.cc/rus/ushakov/426566.html
3,ПАПЕРТЬ,крытая площадка,Крытая площадка перед входом в церковь.,https://how-to-all.com/значение:паперть
4,ПАПЕРТЬ,закрытая площадка,"Притвор — передняя часть церкви, следующая сра...",http://drevne-rus-lit.niv.ru/drevne-rus-lit/te...


In [3]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [4]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[2]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[1]))
        print(prestr(val[2]))

внешний притвор
Па́перть, или внешний притвор (греч. ἄτριον, лат. Atrium, impluvium, pars aperta) — непокрытая кровлей площадка перед внутренним притвором храма, на которой в первые века христианства стояли плачущие и кающиеся. 
площадка перед церковью
Паперть — площадка перед церковью и крыльцо.
церковное крыльцо
Церковное крыльцо, площадка перед входом в церковь.
крытая площадка
 Крытая площадка перед входом в церковь.
закрытая площадка
Притвор — передняя часть церкви, следующая сразу после паперти (паперть — закрытая площадка перед входом в церковь ).
род пистолета
Парабеллума, м. (от латин. Para bellum - готовься к войне). Род автоматического скорострельного пистолета.
самозарядный пистолет
Автоматический самозарядный пистолет Георга Люгера, более известный под названием «Парабеллум» (от лат. Para bellum — готовься к войне), стал легендарным немецким оружием начала XX века.
оружие
«Парабеллум» - легендарный германский пистолет, о котором слышали многие, оружие, по праву ставшее сим

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
p = morph.parse('какой-либо')[0]
p.tag.POS

'ADJF'

In [6]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [7]:
from yargy import rule, or_

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN'),
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

START_N = rule(
    gram('NOUN'),
)

KAK = eq('как')

INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
)

INCLUDING_S = or_(
    eq('называют'),
    eq('называется')
)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
    )

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NPRO')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('NPRO'),
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    
)


Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [8]:
def get_hyperonym(i):
    main_word = (df['HYPONYM'][i]).lower()
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(START, MID, END),
        rule(START_N, INCLUDING_S, END),
        rule(HYPONYM, MID)
)
    parser = Parser(RULE) 
    text = df['TEXT'][i]
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyperonyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyperonyms.append(k)
        print(k)
    print('-----------------------------------------')
    print(' ')

In [9]:
for i in range(len(df)):
    get_hyperonym(i)

Па́перть, или внешний притвор (греч. ἄτριον, лат. Atrium, impluvium, pars aperta) — непокрытая кровлей площадка перед внутренним притвором храма, на которой в первые века христианства стояли плачущие и кающиеся. 
['внешний', 'притвор', '—', 'непокрытая', 'кровлей', 'площадка', 'перед', 'внутренним', 'притвором', 'храма']
-----------------------------------------
 
Паперть — площадка перед церковью и крыльцо.
['паперть', '—', 'площадка', 'перед', 'церковью', 'и', 'крыльцо']
-----------------------------------------
 
Церковное крыльцо, площадка перед входом в церковь.
-----------------------------------------
 
 Крытая площадка перед входом в церковь.
-----------------------------------------
 
Притвор — передняя часть церкви, следующая сразу после паперти (паперть — закрытая площадка перед входом в церковь ).
['притвор', '—', 'передняя', 'часть', 'церкви']
-----------------------------------------
 
Парабеллума, м. (от латин. Para bellum - готовься к войне). Род автоматического скорост

-----------------------------------------
 
Где найти, как и почем приобрести такое пристанище – паркоместо в паркинге или гараж – узнавал Prostobank.ua.
-----------------------------------------
 
На основании информации с указанных сайтов можно заключить, что паркоместа в паркингах Киева на середину мая 2010 года стоят 15-45 тысяч долларов в случае инвестирования в строящийся объект, 30-60 тысяч долларов при покупке паркоместа в готовом объекте.
-----------------------------------------
 
Слишком слаболиквидной недвижимостью являются паркоместа и, тем более, гаражи, чтобы банки брали их в залог.
-----------------------------------------
 
В частности, крупнейшим украинским интернет-стендом объявлений о продаже и аренде гаража и паркоместа на стоянке или в паркинге являются порталы «Авизо-Авто» http://auto.aviso.ua/ и других изданий группы «Авизо».
-----------------------------------------
 
Парку́р — скоростное перемещения и преодоления препятствий с использованием прыжковых элементо

ПАСТОРА'ЛЬ, и, ж. [от латин. Pastoralis — пастушеский]. 1. Произведение (преимущ. Драматическое), идиллически изображающее простую пастушескую жизнь (истор. Лит.).
-----------------------------------------
 
ПАСТОРА'ЛЬ, и, ж. [от латин. Pastoralis — пастушеский] Музыкальное произведение идиллического характера с преобладанием сельских мелодий (муз.).
-----------------------------------------
 
Изучение фундаментальных предметов — фармакологии, патофизиологии и патанатомии — сочеталось с работой в терапевтической и хирургической клиниках.
['фундаментальных', 'предметов', '—', 'фармакологии']
['и', 'патанатомии', '—', 'сочеталось', 'с', 'работой', 'в', 'терапевтической', 'и']
-----------------------------------------
 
В общих чертах патофизиологию можно определить как учение о болезненных явлениях или науку о жизнедеятельности больного организма.
-----------------------------------------
 
В общих чертах патофизиологию можно определить как учение о болезненных явлениях или науку о жизне